<a href="https://colab.research.google.com/github/enriquegiottonini/TAIA-2024/blob/main/1_RNN_LSTM_Vanilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from time import perf_counter

Vamos entonces a desarrollar la función de alimentación a adelante de una RNN

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [ ]:
def forward_V_RNN(inputs, weights):
    # Forward propagation para una RNN vanilla
    x_t, h_t_prev = inputs
    w_hh, w_xh, b_h = weights
    z_t = w_hh @ h_t_prev + w_xh @ x_t + b_h
    h_t = sigmoid(z_t)
    return h_t

In [ ]:
# Data
neurons_hidden = 2   # Dimensión del vector de variables ocultas
neurons_input = 3   # Dimensión del vector de entrada

w_hx = np.random.randn(neurons_hidden, neurons_input)
w_hh = np.random.randn(neurons_hidden, neurons_hidden)
x_t = np.random.randn(neurons_input,1)
b_h = np.random.randn(neurons_hidden, 1)
h_t_prev = np.random.randn(neurons_hidden,1)

# Aplicando un solo paso
h_t = forward_V_RNN([x_t, h_t_prev], [w_hh, w_hx, b_h])

print(f"{h_t=}")

h_t=array([[0.80749125],
       [0.00234686]])


## RNN tipo LSTM

Una LST es un modelo como el que se muestra en la figura, con todo y sus ecuaciones

![](https://github.com/mcd-unison/pln/blob/main/labs/RNN/LSTM.jpg?raw=1)

Como podemos ver tenemos 3 vectores de entrada a la celda:

- $h^{<t-1>}$ el vector de variables ocultas provenientes de un paso anterior,
- $C^{<t-1>}$ el vector de valores de celda (memoria de largo plazo) provenientes de un paso anterior,
- $x^{<t>}$ el vector de variables de entrada. Idealmente debería estar normalizado entre -1 y 1 cada uno de los valores de entrada.

Como podemos ver tenemos varias operaciones:

- Una compuerta de olvido $f$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda anterior (memoria de largo plazo)

- Una compuerta de entrada $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener la activación de la celda actual (memoria de corto plazo)

- Una compuerta de salida $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda actual en el valor de la de la variable oculta correspondiente.

- El calculo de la activación actual, que depende de $h^{<t-1>}$ y $x^{<t>}$, el cual se hace con una tangente hiperbólica, para mantener los valores entre -1 y 1.


Hagamos entonces una celda LSTM


In [ ]:
def forward_LSTM(inputs, weights):
    Xt, ht_prev, Ct_prev = inputs
    Ui, Wi, Uf, Wf, Uo, Wo, U, W = weights

    i = sigmoid(Xt @ Ui + ht_prev @ Wi) # Compuerta de entrada
    f = sigmoid(Xt @ Uf + ht_prev @ Wf) # Compuerta de olvido
    o = sigmoid(Xt @ Uo + ht_prev @ Wo) # Compuerta de salida
    C_t_short = np.tanh(Xt @ U + ht_prev @ W)  # Memoria de corto plazo
    C_t = sigmoid(f * Ct_prev + i * C_t_short) # Memoria de corto y largo plazo

    h_t = np.tanh(C_t) * o
    return h_t, C_t

Vamos a probar como funciona

In [ ]:
# Data
nh = 2   # Dimensión del vector de variables ocultas
nx = 3   # Dimensión del vector de entrada
neurons_hidden = 2
neurons_input = 3

Ui = np.random.randn(neurons_input, neurons_hidden)
Wi = np.random.randn(neurons_hidden, neurons_hidden)

Uf = np.random.randn(neurons_input, neurons_hidden)
Wf = np.random.randn(neurons_hidden, neurons_hidden)

Uo = np.random.randn(neurons_input, neurons_hidden)
Wo = np.random.randn(neurons_hidden, neurons_hidden)

U = np.random.randn(neurons_input, neurons_hidden)
W = np.random.randn(neurons_hidden, neurons_hidden)

h_t_prev = 2 * np.random.standard_normal((1, neurons_hidden)) - 1
C_t_prev = np.random.standard_normal((neurons_hidden,1))
x_t = 2 * np.random.standard_normal((1,neurons_input)) - 1

# Aplicando un solo paso
h_t, C_t = forward_LSTM(
    [x_t, h_t_prev, C_t_prev],
    [Ui, Wi, Uf, Wf, Uo, Wo, U, W]
)
print(f"{h_t=}\n\n{C_t=}")

h_t=array([[0.27287196, 0.00031408],
       [0.27284445, 0.00030528]])

C_t=array([[0.71848709, 0.38409632],
       [0.71838702, 0.37229156]])


## La función `scan`para el cálculo de BPTT

La función `scan` se usa para calcular la propagación hacia adelante. Si la funcións e implementa en un *framework* como *Tensorflow* o *pytorch*, entonces se puede ir guardando los gradientes de cada aplicación a lo largo del tiempo y usarlos en el calculo del gradiente para la función de aprendizaje.

Aquí solo vamos a tratar de mostrar como funcionaría dicha función, la cual recibe:

- `elems` : lista de entradas (`X`)
- `weights` : los parámetros que necesita la función de feedforward para su cálculo (pesos)
- `h_0` : estado oculto inicial

`scan` va por todos los valores de `x` en `elems`, llama la función de feedforward con los argumentos necesarios, guarda el estado oculto `h_t` y agrega el valor de `h_t` a una lista.

Vamos a hacer la función de scan para una celda tipo RNN vainilla

In [ ]:
def scan_V_RNN(elems, weights, h_0): # Forward propagation for RNNs
    h_t = h_0
    h = []
    for i, x in enumerate(elems):
        h_t = forward_V_RNN([x, h_t], weights)
        h.append(h_t)
    return h, h_t

Vamos a probar inicializando una posible red RNN vainilla en un probable pornblema de PLN

In [ ]:
np.random.seed(10)

emb = 128                       # Embedding
T = 256                         # Tamaño de secuencia de tokens
h_dim = 16                      # Estados ocultos

h_0 = np.zeros((h_dim, 1))      # Estado inicial

# Inicialización aleatoria de pesos y sesgos
Whh = np.random.standard_normal((h_dim, h_dim))
Wxh = np.random.standard_normal((h_dim, emb))
bh = np.random.standard_normal((h_dim, 1))

# Inicialización aleatoria de una secuencia de tokens (en embeddings)
X = np.random.standard_normal((T, emb))

weights = [Whh, Wxh, bh]

In [ ]:
# vanilla RNNs
tic = perf_counter()
h, h_T = scan_V_RNN(X, weights, h_0)
toc = perf_counter()
RNN_time=(toc-tic)*1000
print (f"Tomó {RNN_time:.2f}ms ejecutar el método de RNN vainilla.")

Tomó 6.88ms ejecutar el método de RNN vainilla.


**Desarrolla la función de scan para LSTM y prueba con la

---

misma secuencia de entradas para una red LSTM**

In [ ]:
def scan_LSTM(elems, weights, h0, C0):
  ht = h0
  Ct = C0
  h = []
  for x in elems:
    ht, Ct = forward_LSTM([x, ht, Ct], weights)
    h.append(ht)
  return h, ht

In [ ]:
np.random.seed(10)

emb = 128                       # Embedding
T = 256                         # Tamaño de secuencia de tokens
h_dim = 16                      # Estados ocultos

# Inicialización aleatoria de pesos y sesgos
h_0 = np.zeros((1, h_dim))      # Estado inicial
C_0 = np.zeros((h_dim, 1))
Ui = np.random.randn(emb, h_dim)
Wi = np.random.randn(h_dim, h_dim)
Uf = np.random.randn(emb, h_dim)
Wf = np.random.randn(h_dim, h_dim)
Uo = np.random.randn(emb, h_dim)
Wo = np.random.randn(h_dim, h_dim)
U = np.random.randn(emb, h_dim)
W = np.random.randn(h_dim, h_dim)

# Inicialización aleatoria de una secuencia de tokens (en embeddings)
X = np.random.standard_normal((T, emb))

weights = [Ui, Wi, Uf, Wf, Uo, Wo, U, W]

In [ ]:
tic = perf_counter()
h, h_T = scan_LSTM(X, weights, h_0, C_0)
toc = perf_counter()
LSTM_time=(toc-tic)*1000
print (f"Tomó {LSTM_time:.2f}ms ejecutar el método de RNN vainilla.")

Tomó 29.52ms ejecutar el método de RNN vainilla.
